In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

# Step 1: Load and Preprocess Data
data = pd.read_csv('time_series_data.csv')  # Replace with your dataset path
X = data['feature'].values
y = data['target'].values

# Normalize the data
X = (X - np.mean(X)) / np.std(X)

# Step 2: Handling Class Imbalance (Optional)
# You can apply over-sampling or under-sampling to handle class imbalance.
# Example for over-sampling:
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X.reshape(-1, 1), y)

# Example for under-sampling:
rus = RandomUnderSampler(random_state=42)
X_resampled, y_resampled = rus.fit_resample(X.reshape(-1, 1), y)

# Step 3: Split the Data into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Step 4: Build the RNN Model
model = Sequential([
    SimpleRNN(64, input_shape=(X_train.shape[1], 1)),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Step 5: Hyperparameter Tuning
# You can perform hyperparameter tuning to optimize the model's performance.

# Example of hyperparameter tuning:
# hyperparameters = {'batch_size': [16, 32, 64], 'epochs': [10, 20, 30]}
# best_accuracy = 0
# best_model = None
# for batch_size in hyperparameters['batch_size']:
#     for epochs in hyperparameters['epochs']:
#         tuned_model = Sequential([
#             SimpleRNN(64, input_shape=(X_train.shape[1], 1)),
#             Dense(1, activation='sigmoid')
#         ])
#         tuned_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#         history = tuned_model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_split=0.2, verbose=0)
#         accuracy = max(history.history['val_accuracy'])
#         if accuracy > best_accuracy:
#             best_accuracy = accuracy
#             best_model = tuned_model
# model = best_model  # Use the best-tuned model

# Step 6: Train the Model
history = model.fit(X_train, y_train, batch_size=64, epochs=10, validation_split=0.2)

# Step 7: Evaluate the Model
y_pred = (model.predict(X_test) > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
print(f'Test Accuracy: {accuracy:.4f}')
print(classification_report(y_test, y_pred))
conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

# Step 8: Plot Accuracy and Loss Curves
plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Step 9: Predict for Sample Data
sample_data = np.array([0.1, 0.2, 0.3, 0.4])  # Replace with your own sample data
sample_data = (sample_data - np.mean(sample_data)) / np.std(sample_data)  # Normalize the sample data
sample_data = sample_data.reshape(1, -1, 1)
prediction = model.predict(sample_data)
predicted_class = 'Positive' if prediction > 0.5 else 'Negative'
print(f'Sample Data: {sample_data[0, :, 0]}')
print(f'Predicted Class: {predicted_class}')
